# Libraries

In [1]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
from scipy.stats import t as t_student
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
### plotting
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import cvxpy as cp
import time
### Other
import itertools
from timeit import default_timer as timer
from itertools import product
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings("ignore")

In [2]:
### QMC
!pip install gpytorch
!pip install qmcpy
import qmcpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for qmcpy: filename=qmcpy-1.4.5-cp310-cp310-linux_x86_64.whl size=7693205 sha256=b3136b4e31cbaeafb3565db25ecb99d4ee22a1bbd9a9ea6b76bc7a14b04d4486
  Stored in directory: /root/.cache/pip/wheels/ac/82/9f/b755d059c498da7517193ae6e58198d71e1089d14b4713733e
Successfully built qmcpy


In [41]:
import numpy as np
import scipy.stats as stats
import scipy.linalg as la
import qmcpy
from scipy.stats import multivariate_t, chi2, norm

def t_student_pdf(x, sigma_IS):
  return t_student.pdf( x =  x, df = nu_IS, loc = 0, scale = sigma_IS)

def t_student_ppf(x, sigma_IS):
  return t_student.ppf( q =  x, df = nu_IS, loc = 0, scale = sigma_IS)


def covariance_matrix(sigma, rho):
    """Compute the covariance matrix.
    Args:
    - sigma (array): Array of volatilities of each stock.
    - rho (array): Correlation matrix.
    Returns:
    - SIGMA (array): Covariance matrix.
    """
    sigma = np.diag(sigma)  # Diagonal matrix of volatilities
    SIGMA = np.dot(sigma, np.dot(rho, sigma))  # Covariance matrix calculation
    return SIGMA

def VG_characteristic_function(u, SIGMA, T, r, theta, nu):
    """Calculate the characteristic function of Variance-Gamma process.
    Args:
    - u (array): Vector in Rd.
    - SIGMA (array): Covariance matrix.
    - T (float): Terminal time.
    - r (float): Short rate.
    - theta (array): Array of theta values.
    - nu (float): Nu parameter.
    Returns:
    - phi (complex): Characteristic function value.
    """
    d = len(theta)  # Number of stocks
    w = (1/nu) * np.log(1 - nu * theta - 0.5 * nu * np.diag(SIGMA))  # Martingale correction term
    phi = np.exp(np.multiply(1j * T, np.dot(r + w, u))) * (1 - np.multiply(1j * nu, np.dot(theta, u)) +
                                                           0.5 * nu * np.dot(u, np.dot(SIGMA, u))) ** (-T/nu)
    return phi

def fourier_payoff_call_on_min(u):
    """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
    Args:
    - u (array): Array of Fourier frequencies.
    Returns:
    - payoff (float): Call on min option payoff Fourier transofrm value.
    """
    denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
    return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_VG_call_on_min(R):
    """Calculate the integrand for QMC estimation of the rainbow option under VG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
    p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def integrand_to_optimize_VG_basket_put(R):
    """Calculate the integrand for QMC estimation of the basket put option under VG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, d*K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
    p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def RQMC_fourier_VG_call_on_min_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, m, nu_IS, sigma_IS):
  """Price of call on min option under the multivariate VG model using RQMC
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  - theta (array): Array of theta values.
  - nu (float): Nu parameter.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of QMC points.
  - m (int): Number of randomizations of RQMC (digital shifts).
  - R (array): Array of damping parameters.
  - nu_IS (float): Nu parameter of the IS t-Student distribution.
  - sigma_IS (array): Scale parameter of the IS t-Student distribution.
  Returns:
  - qmc_estimate (float): QMC estimate of option price.
  - qmc_stat_error (float): QMC statistical error (absolute not relative).
  """
  d = len(S0) #number of stocks
  X0 = np.log(np.divide(S0,K)) #element-wise division
  discount=((2*np.pi)**(-d))*np.exp(-r*T)*np.exp(-R @ X0) #modified discount factor
  V_list  = np.zeros(m) #Contains m price estimates each corresponding to a shifted sobol sequence
  for i in range(m):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode = True, randomize='DS', seed = i).gen_samples(N)
    xi_sobol_scrambled_mapped = np.asarray([t_student_ppf(row, param) for row, param in zip(zip(*xi_sobol_scrambled), sigma_IS)]).T
    V = 0 #Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N): # for each sobol point
      u = xi_sobol_scrambled_mapped[n] #inverse CDF of uniformly distributed sample to have sample from standard normal
      y = u + np.multiply(1j,R) #Change of variable, shift by damping parameter to integrate over axis // to real line.
      phi = VG_characteristic_function(y,SIGMA,T,r,theta,nu) #evaluation of characteristic function at all sobol points
      p = fourier_payoff_call_on_min(y) # evaluation of Fourier Transformed Payoff function at all sobol points
      par_vect_t_student_pdf = map(lambda x, args: t_student_pdf(x, args), u, sigma_IS)
      IS_pdf_prod = np.prod(np.asarray(list(par_vect_t_student_pdf))) #product of t-student densities evaluated at each component of d-dim vector
      V += (1/N) * K * discount * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
    V_list[i] = np.real(V)
  qmc_estimate = np.mean(V_list) #QMC estimate
  qmc_stat_error  = 1.96*np.std(V_list)/np.sqrt(m) #QMC statistical error
  return qmc_estimate,qmc_stat_error

def RQMC_fourier_VG_basket_put_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, m, nu_IS, sigma_IS):
  """Price of call on min option under the multivariate VG model using RQMC
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  - theta (array): Array of theta values.
  - nu (float): Nu parameter.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of QMC points.
  - m (int): Number of randomizations of RQMC (digital shifts).
  - R (array): Array of damping parameters.
  - nu_IS (float): Nu parameter of the IS t-Student distribution.
  - sigma_IS (array): Scale parameter of the IS t-Student distribution.
  Returns:
  - qmc_estimate (float): QMC estimate of option price.
  - qmc_stat_error (float): QMC statistical error (absolute not relative).
  """
  d = len(S0) #number of stocks
  X0 = np.log(np.divide(S0,d*K)) #element-wise division
  discount=((2*np.pi)**(-d))*np.exp(-r*T)*np.exp(-R @ X0) #modified discount factor
  V_list  = np.zeros(m) #Contains m price estimates each corresponding to a shifted sobol sequence
  for i in range(m):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode = True, randomize='DS', seed = i).gen_samples(N)
    xi_sobol_scrambled_mapped = np.asarray([t_student_ppf(row, param) for row, param in zip(zip(*xi_sobol_scrambled), sigma_IS)]).T
    V = 0 #Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N): # for each sobol point
      u = xi_sobol_scrambled_mapped[n] #inverse CDF of uniformly distributed sample to have sample from standard normal
      y = u + np.multiply(1j,R) #Change of variable, shift by damping parameter to integrate over axis // to real line.
      phi = VG_characteristic_function(y,SIGMA,T,r,theta,nu) #evaluation of characteristic function at all sobol points
      p = fourier_payoff_basket_put(y) # evaluation of Fourier Transformed Payoff function at all sobol points
      par_vect_t_student_pdf = map(lambda x, args: t_student_pdf(x, args), u, sigma_IS)
      IS_pdf_prod = np.prod(np.asarray(list(par_vect_t_student_pdf))) #product of t-student densities evaluated at each component of d-dim vector
      V += (1/N) * K * discount * np.exp(1j * u @ X0) * phi * p / IS_pdf_prod
    V_list[i] = np.real(V)
  qmc_estimate = np.mean(V_list) #QMC estimate
  qmc_stat_error  = 1.96*np.std(V_list)/np.sqrt(m) #QMC statistical error
  return qmc_estimate,qmc_stat_error


def chi_pdf(u, nu):
    """Compute the probability density function of the chi distribution.
    Args:
    - u (float): Value to evaluate the PDF at.
    - nu (float): Degrees of freedom parameter.
    Returns:
    - pdf (float): PDF value at u.
    """
    return 2 ** (-0.5 * nu + 1) * u ** (nu - 1) * np.exp(-0.5 * u ** 2) / scipy.special.gamma(0.5 * nu)

def LagRQMC_fourier_VG_call_on_min_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, M_lag, nu_IS, SIGMA_IS):
  """Perform Laguerre-RQMC hybrid quadrature estimate for call on min options under the VG distribution with multivariate domain transformation
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  - theta (array): Array of theta values.
  - nu (float): Nu parameter.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of QMC points.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - nu_IS (float): Nu parameter of the t-Student transformation distribution.
  - SIGMA_IS (array): Covariance matrix of the IS t-Student transformation distribution.
  Returns:
  - V (float): Option price estimate.
  """
  d = len(S0) #number of stocks
  X0 = np.log(np.divide(S0,K)) #element-wise division
  discount= ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0) #modified discount factor
  SIGMA_IS_tilde = nu_IS * SIGMA_IS  # scaling the normal disitribution by sqrt(nu_IS)
  L_IS = la.cholesky(SIGMA_IS_tilde)
  outer_integral_values = np.zeros(M_lag)
  [lag_abcissas,lag_weights] = np.polynomial.laguerre.laggauss(M_lag) #Laguerre quadrature nodes and weights.
  for k in range(M_lag):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode = True, randomize='DS', seed = 1).gen_samples(N)
    xi_sobol_mapped = norm.ppf( q = xi_sobol_scrambled, loc = 0, scale = 1) # sampels from N(0,I_d)
    inner_integral = 0 #Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N): # number of QMC points
      u = L_IS @ xi_sobol_mapped[n] / lag_abcissas[k] # sample from  sqrt(nu_IS) * N(O, Sigma_IS) divided by y(k)
      y = u + np.multiply(1j,R) #Change of variable, shift by damping parameter to integrate over axis // to real line.
      phi = VG_characteristic_function(y ,SIGMA,T,r,theta,nu) #evaluation of characteristic function at all sobol points
      p = fourier_payoff_call_on_min(y) # evaluation of Fourier Transformed Payoff function at all sobol points
      IS_pdf_prod  = multivariate_t.pdf(x = u , df = nu_IS, loc = np.zeros(d), shape = SIGMA_IS)
      inner_integral_increment = (1/N) * K * discount * np.real( np.exp(1j * u @ X0 ) * phi * p ) / IS_pdf_prod
      if (inner_integral_increment - inner_integral_increment != 0):
        inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * chi_pdf(lag_abcissas[k],nu_IS) / np.exp(- lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V

def LagRQMC_fourier_VG_basket_put_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, M_lag, nu_IS, SIGMA_IS):
  """Perform Laguerre-RQMC hybrid quadrature estimate for call on min options under the VG distribution with multivariate domain transformation
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Risk-free interest rate.
  - T (float): Time to maturity.
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  - theta (array): Array of theta values.
  - nu (float): Nu parameter.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of QMC points.
  - R (array): Array of damping parameters.
  - M_lag (int): Number of Laguerre quadrature nodes.
  - nu_IS (float): Nu parameter of the t-Student transformation distribution.
  - SIGMA_IS (array): Covariance matrix of the IS t-Student transformation distribution.
  Returns:
  - V (float): Option price estimate.
  """
  d = len(S0) #number of stocks
  X0 = np.log(np.divide(S0,d*K)) #element-wise division
  discount= ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0) #modified discount factor
  SIGMA_IS_tilde = nu_IS * SIGMA_IS  # scaling the normal disitribution by sqrt(nu_IS)
  L_IS = la.cholesky(SIGMA_IS_tilde)
  outer_integral_values = np.zeros(M_lag)
  [lag_abcissas,lag_weights] = np.polynomial.laguerre.laggauss(M_lag) #Laguerre quadrature nodes and weights.
  for k in range(M_lag):
    xi_sobol_scrambled = qmcpy.DigitalNetB2(d, graycode = True, randomize='DS', seed = 1).gen_samples(N)
    xi_sobol_mapped = norm.ppf( q = xi_sobol_scrambled, loc = 0, scale = 1) # sampels from N(0,I_d)
    inner_integral = 0 #Initialisation of Option price contract value for ith rQMC iteration
    for n in range(N): # number of QMC points
      u = L_IS @ xi_sobol_mapped[n] / lag_abcissas[k] # sample from  sqrt(nu_IS) * N(O, Sigma_IS) divided by y(k)
      y = u + np.multiply(1j,R) #Change of variable, shift by damping parameter to integrate over axis // to real line.
      phi = VG_characteristic_function(y ,SIGMA,T,r,theta,nu) #evaluation of characteristic function at all sobol points
      p = fourier_payoff_basket_put(y) # evaluation of Fourier Transformed Payoff function at all sobol points
      IS_pdf_prod  = multivariate_t.pdf(x = u , df = nu_IS, loc = np.zeros(d), shape = SIGMA_IS)
      inner_integral_increment = (1/N) * K * discount * np.real( np.exp(1j * u @ X0 ) * phi * p ) / IS_pdf_prod
      if (inner_integral_increment - inner_integral_increment != 0):
        inner_integral_increment = 0
      inner_integral += inner_integral_increment
    outer_integral_values[k] = inner_integral * lag_weights[k] * chi_pdf(lag_abcissas[k],nu_IS) / np.exp(- lag_abcissas[k])
  V = np.sum(outer_integral_values)
  return V

# Call on min options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [14]:
# Model and payoff parameters
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def call_on_min_constraint_1(R):
    return -1 * R

def call_on_min_constraint_2(R):
    return -1 - np.sum(R)

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
        {'type': 'ineq', 'fun': call_on_min_constraint_1},
        {'type': 'ineq', 'fun': call_on_min_constraint_2},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-5.83641532 -5.83637836 -5.83641692]


## Pricing using RQMC in the Fourier space

### Univariate domain transformation (RQMC)

In [32]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### QMC parameters ###############
m = 30 # number of digital shifts of RQMC
N = 2**6 # number of QMC Sobol points
nu_IS =  2*T / nu - 1
print("nu_IS =",nu_IS)
#sigma_IS = (nu * sigma[0]**2 * nu_IS *0.5)**(T / (nu - 2*T))* nu_IS**( nu / (4*T-2*nu))
sigma_IS = (nu * sigma**2 * nu_IS *0.5)**(T / (nu - 2*T))* nu_IS**( nu / (4*T-2*nu))
print("sigma_IS =", sigma_IS)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate, RQMC_stat_estimate =  RQMC_fourier_VG_call_on_min_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, m, nu_IS, sigma_IS)
print("RQMC estimate =", round(RQMC_estimate,5), ", Relative Statistical Error =", round(RQMC_stat_estimate / RQMC_estimate,5)  )

nu_IS = 9.0
sigma_IS = [7.16254954 7.16254954 7.16254954]
RQMC estimate = 1.79387 , Relative Statistical Error = 0.01492


### Multivariate domain transformation (Laguerre-RQMC)

In [18]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### QMC parameters ###############
N = 2**6 # number of QMC Sobol points
M_lag = 2**6 # Number of laguerre quadrature nodes
nu_IS =  2*T / nu - dimension
print("nu_IS =",nu_IS)
SIGMA_IS  = la.inv(SIGMA)
print("sigma_IS =", sigma_IS)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate =  LagRQMC_fourier_VG_call_on_min_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, M_lag, nu_IS, SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5)  )

nu_IS = 7.0
sigma_IS = 7.162549543128552
RQMC estimate = 1.76882


# Basket Put

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [43]:
# Model and payoff parameters
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
      {'type': 'ineq', 'fun': basket_put_constraint},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [2.21940094 2.21946564 2.21941962]


## Pricing using RQMC in the Fourier space

### Univariate domain transformation (RQMC)

In [48]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### QMC parameters ###############
m = 30 # number of digital shifts of RQMC
N = 2**9 # number of QMC Sobol points
nu_IS =  2*T / nu - 1
print("nu_IS =",nu_IS)
#sigma_IS = (nu * sigma[0]**2 * nu_IS *0.5)**(T / (nu - 2*T))* nu_IS**( nu / (4*T-2*nu))
sigma_IS = (nu * sigma**2 * nu_IS *0.5)**(T / (nu - 2*T))* nu_IS**( nu / (4*T-2*nu))
print("sigma_IS =", sigma_IS)
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate, RQMC_stat_estimate =  RQMC_fourier_VG_basket_put_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, m, nu_IS, sigma_IS)
print("RQMC estimate =", round(RQMC_estimate,5), ", Relative Statistical Error =", round(RQMC_stat_estimate / RQMC_estimate,5)  )

nu_IS = 9.0
sigma_IS = [7.16254954 7.16254954 7.16254954]
RQMC estimate = 6.38646 , Relative Statistical Error = 0.03771


### Multivariate domain transformation (RQMC)

In [47]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### QMC parameters ###############
N = 2**11 # number of QMC Sobol points
M_lag = 2**5 # Number of laguerre quadrature nodes
nu_IS =  2*T / nu - dimension
print("nu_IS =",nu_IS)
SIGMA_IS  = la.inv(SIGMA)
print("sigma_IS =", sigma_IS)
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
RQMC_estimate =  LagRQMC_fourier_VG_basket_put_pricer(S0, K, r, T, sigma, rho, theta, nu, SIGMA, N, R, M_lag, nu_IS, SIGMA_IS)
print("RQMC estimate =", round(RQMC_estimate,5)  )

nu_IS = 7.0
sigma_IS = [7.16254954 7.16254954 7.16254954]
RQMC estimate = 6.568
